In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from constants import *
import os
from eda_utils import *
from utils import eval_stat
import seaborn as sns
import pickle as pkl

c:\Users\malay\miniconda3\envs\cs776\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_pickle(bihar_data_419)

In [4]:
df

,timestamp,device_id,block,district,latitude,longitude,rh,temp,pm25
0,2023-06-01 00:00:00,1201230029,ARARIA,ARARIA,26.148,87.457,NaN,NaN,NaN
1,2023-06-01 00:00:00,1201230033,KURSAKANTA,ARARIA,26.359,87.443,NaN,NaN,NaN
2,2023-06-01 00:00:00,1201230034,SIKTY,ARARIA,26.407,87.549,NaN,NaN,NaN
3,2023-06-01 00:00:00,1201230052,NARPATGANJ,ARARIA,26.338,87.107,NaN,NaN,NaN
4,2023-06-01 00:00:00,1201230053,BHARGAMA,ARARIA,26.089,87.152,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
930931,2023-08-18 23:00:00,B4E62D2A3BA4,LAURIYA,WCHAMPARAN,26.994,84.410,NaN,NaN,NaN
930932,2023-08-18 23:00:00,B4E62D2A46CD,BAGAHA-II,WCHAMPARAN,27.133,84.060,NaN,NaN,NaN
930933,2023-08-18 23:00:00,BCDDC247BFE3,BHITAHA,WCHAMPARAN,26.907,84.136,NaN,NaN,NaN
930934,2023-08-18 23:00:00,DC4F22364445,PIPRASI,WCHAMPARAN,27.060,84.023,NaN,NaN,NaN


In [5]:
grp = df.groupby('timestamp')
df = df[['timestamp', 'latitude', 'longitude', 'rh', 'temp', 'pm25']]
len(grp)

1896

In [6]:
timestamps = []

In [7]:
cnt = 0

for name, group in grp:
    cnt += (1 if group['pm25'].count() >= 300 else 0)
    if group['pm25'].count() >= 300:
        timestamps.append(name)
cnt

541

In [8]:
# timestamps

In [9]:
data = df[df['timestamp'].isin(timestamps)]

In [10]:
file_name = f'{data_bihar}/Bihar_419_data_modified.csv'
data.to_csv(file_name, header=True, index=False)

In [11]:
data.groupby('timestamp').count()

,latitude,longitude,rh,temp,pm25
timestamp,,,,,
2023-07-21 17:00:00,491,491,301,287,300
2023-07-21 18:00:00,491,491,298,292,300
2023-07-21 19:00:00,491,491,297,293,300
2023-07-21 20:00:00,491,491,292,292,300
2023-07-22 10:00:00,491,491,299,291,300
...,...,...,...,...,...
2023-08-16 08:00:00,491,491,354,344,355
2023-08-16 09:00:00,491,491,374,338,373
2023-08-16 10:00:00,491,491,379,322,374


In [12]:
# df['timestamp'] = df['timestamp'].values.astype(float)
data['timestamp'] = data['timestamp'].values.astype(float)

c:\Users\malay\miniconda3\envs\cs776\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
data.dtypes

timestamp    float64
latitude     float64
longitude    float64
rh           float64
temp         float64
pm25         float64
dtype: object

In [14]:
data = data.to_numpy()
data.shape

(265631, 6)

In [18]:
def impute(data, method):
    assert method == 'knn' or method == 'mean' or method == 'iterative', 'method can only knn, mean or iterative'
    # KNN Imputer
    if method == 'knn':
        imputer = KNNImputer(n_neighbors=2)
    # Mean Imputer
    elif method == 'mean':
        imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    # Iterative Imputer
    elif method == 'iterative':
        imputer = IterativeImputer(random_state=0)
    
    return imputer.fit_transform(data)

def stat(data, method='knn'):
    
    stat_data = []
        
    imputed_data = impute(data, method=method)
    X_train, X_test, y_train, y_test = train_test_split(imputed_data[:, :-1], imputed_data[:, -1], test_size=0.33)
    
    X_train, X_test, y_train, y_test = np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

    model = XGBRegressor(objective ='reg:squarederror')
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_stat = eval_stat(y_train_pred, y_train)
    test_stat = eval_stat(y_test_pred, y_test)

    model_name = model_dir + f"bihar_xgb_{method}.pkl"
    pkl.dump(model, open(model_name, "wb"))

    stat_data.append({'Train_RMSE': train_stat[0], 'Train_Pearson_R': train_stat[1], \
                    'Test_RMSE': test_stat[0], 'Test_Pearson_R': test_stat[1]})
    
    return stat_data

In [22]:
print(stat(data, 'iterative'))

[{'Train_RMSE': 7.681376262717706, 'Train_Pearson_R': 0.859975267564746, 'Test_RMSE': 8.42375213125643, 'Test_Pearson_R': 0.845933573328844}]
